#### Initialization ####

Contains ONLY functions, "default" package imports, and environment settings.

In [ ]:
print("Initializing project...")

In [ ]:
import warnings
warnings.filterwarnings("ignore")

#### Package installer ####

Install a package if not already present.

In [ ]:
import sys
import subprocess
import pkg_resources

def install_if_needed(package):
    try:
        pkg_resources.get_distribution(package)
        print(f"{package} is already installed.")
    except pkg_resources.DistributionNotFound:
        print(f"{package} not found. Installing...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])

In [ ]:
# Wrap long labels.
import textwrap
def split_label(label, max_length=25):
    return textwrap.fill(label, width=max_length)

#### Default packages ###

Load most-commonly used packages and install if not already present.

In [ ]:
# Contributed libraries
install_if_needed('pandas')
import pandas as pd

install_if_needed('matplotlib')
import matplotlib.pyplot as plt

install_if_needed('networkx')
import networkx as nx

install_if_needed('numpy')
import numpy as np

# Standard libraries
import os

#### Environment ####

In [ ]:
pd.options.display.max_columns = None # Do not truncate numerous columns.

#### Done ####

In [ ]:
print("...initialization complete.")